<a href="https://colab.research.google.com/github/GazalaSayyad/Dash-in-Python/blob/master/User_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dash 

In [ ]:
!pip install dash==0.21.1  
!pip install dash-renderer==0.13.0  
!pip install dash-html-components==0.11.0
!pip install dash-core-components==0.23.0  
!pip install plotly --upgrade
!pip install dash-auth

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.9.0)
     |████████████████████████████████| 471kB 2.7MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 1.2MB 9.0MB/s 
     |████████████████████████████████| 3.4MB 17.9MB/s 
     |████████████████████████████████| 194kB 30.4MB/s 
     |████████████████████████████████| 1.8MB 36.3MB/s 
  Created wheel for dash-auth: filename=dash_auth-1.4.1-cp36-none-any.whl size=476154 sha256=19bb27a893050ad500d4c218447a1f60d2ae9ca702bab40906fb175266ce3238
  Stored in directory: /root/.cache/pip/wheels/9b/6d/64/a95a7e2050ba581161fcf27f862806368bbc34b998e6bc2d37
  Created wheel for flask-seasurf: filename=Flask_SeaSurf-0.2.2-cp36-none-any.whl size=5562 sha256=8a9180317c5809b948c47ba21a0519226d8bf3c9ea5f1e0af9dfa5ffef918807
  Stored in directory: /root/.cache/pip/wheels/75/4a/8e/ca53c401e38aa1e0e6cd626c9bc3ef5230e362e3

In [ ]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.1M  100 13.1M    0     0  4847k      0  0:00:02  0:00:02 --:--:-- 4845k
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [ ]:
import pandas as pd
df = pd.read_excel("Service Provider Daily Report (Responses).xlsx")
df

,Timestamp,User Id,Date,Starting Time,Leaving Time,Total Sales today(in 1000),Total Profit Earned
0,2020-08-27 15:01:48.855,SP0001,2020-08-27,10:30:00,18:30:00,15,4000
1,2020-08-27 15:10:21.990,SP1204,2020-08-27,09:00:00,07:00:00,7,2000
2,2020-08-27 16:52:20.206,SP8468,2020-08-27,20:51:00,05:51:00,9000,4000
3,2020-08-27 17:08:24.412,SP1358,2020-08-14,03:00:00,21:30:00,6000,3000
4,2020-08-27 17:08:58.801,SP4693,2020-08-17,02:20:00,15:30:00,8000,3000
5,2020-08-27 17:09:30.841,SP9853,2020-08-23,01:30:00,17:00:00,9000,5000
6,2020-08-27 17:10:37.288,SP0762,2020-08-23,01:00:00,21:00:00,6500,4500


In [ ]:
df = df.drop(columns=['Timestamp', 'Starting Time','Leaving Time'])
df

,User Id,Date,Total Sales today(in 1000),Total Profit Earned
0,SP0001,2020-08-27,15,4000
1,SP1204,2020-08-27,7,2000
2,SP8468,2020-08-27,9000,4000
3,SP1358,2020-08-14,6000,3000
4,SP4693,2020-08-17,8000,3000
5,SP9853,2020-08-23,9000,5000
6,SP0762,2020-08-23,6500,4500


In [ ]:
pv =pd.pivot_table(df,index=["User Id"])

In [ ]:
trace1 = go.Bar(x=pv.index,y=pv[('Total Sales today(in 1000)')],name='Total Sales today(in 1000)')
trace2 = go.Bar(x=pv.index, y=pv[('Total Profit Earned')], name='Total Profit Earned')



In [ ]:

%%writefile dash_app.py
import dash
import dash_auth
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd


VALID_USERNAME_PASSWORD_PAIRS = {
    'SSS': 'india'
}

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__)#, external_stylesheets=external_stylesheets)
auth = dash_auth.BasicAuth(
    app,
    VALID_USERNAME_PASSWORD_PAIRS
)

df = pd.read_excel("Service Provider Daily Report (Responses).xlsx")
pv = pd.pivot_table(df, index=['User Id'],values=['Total Sales today(in 1000)','Total Profit Earned'])
trace1 = go.Bar(x=pv.index,y=pv[('Total Sales today(in 1000)')],name='Total Sales today(in 1000)')
trace2 = go.Bar(x=pv.index, y=pv[('Total Profit Earned')], name='Total Profit Earned')
app.layout = html.Div(children=[
    html.H1(children='Sales and Profit Earned Report'),
    html.Div(children='''Sales Report.'''),
    dcc.Graph(
        id='example-graph',
        figure={
            'data': [trace1, trace2],
            'layout':
            go.Layout(title='SSS User', barmode='stack')
        })
])
if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting dash_app.py


In [ ]:
# launch ngrok
get_ipython().system_raw('./ngrok http 8050 &')

In [ ]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://c59516074107.ngrok.io


In [ ]:
!python dash_app.py